### --- Day 16: The Floor Will Be Lava ---

Puzzle description redacted as-per Advent of Code guidelines

You may find the puzzle description at: https://adventofcode.com/2023/day/16

In [2]:
#!import ../Utils.ipynb

In [3]:
var inputLines = LoadPuzzleInput(2023, 16);
WriteLines(inputLines);

Loading puzzle file: Day16.txt
Total lines: 110
Max line length: 110

\...|.........\........./.................\...\.../.\/../|....\................./..../........-\..-.\-..../...
...|....\................../.............../-....--..........-......-......\....-........../-..............|..
..............\....../.........../...................|.........\.................|.-................../.......
.........................\........../..-........./.../..........|....................-....\./.........|.......
.-.......\......-.-................|......................................-..............\......./............


In [4]:
class MirrorMap(string[] inputLines) {
    private string[] inputLines = inputLines;

    public int Rows = inputLines.Length;
    public int Cols = inputLines[0].Length;

    public bool IsValid(Point p) {
        return 0 <= p.row && p.row < Rows && 0 <= p.col && p.col < Cols;
    }

    public char Char(Point p) => inputLines[p.row][p.col];
}

record Point(int row, int col) {
    public Point Up() => new(row - 1, col);
    public Point Down() => new(row + 1, col);
    public Point Left() => new(row, col - 1);
    public Point Right() => new(row, col + 1);
}

enum Direction {
    Up,
    Down,
    Left,
    Right
}

record BeamPoint(Point point, Direction direction);

In [5]:
HashSet<Point> ScanAll(MirrorMap map, BeamPoint startPoint)
{
    var queue = new Queue<BeamPoint>();
    queue.Enqueue(startPoint);

    var safety = 0;

    HashSet<BeamPoint> scannedPoints = new();
    while (queue.TryDequeue(out var beamPoint)) {
        if (safety++ >= 100000) {
            throw new Exception("Safety limit exceeded");
        }

        var (point, direction) = beamPoint;

        if (!map.IsValid(point)) {
            continue;
        }
        if (scannedPoints.Contains(beamPoint)) {
            continue;
        }

        scannedPoints.Add(beamPoint);

        var ch = map.Char(point);
        foreach (var next in GetNext(beamPoint, ch)) {
            queue.Enqueue(next);
        }
    }

    return scannedPoints.Select(sp => sp.point).ToHashSet();
}

HashSet<Point> ScanAll(MirrorMap map) => ScanAll(map, new(new(0, 0), Direction.Right));

IList<BeamPoint> GetNext(BeamPoint bp, char ch) => (bp.direction, bp.point, ch) switch {
    (Direction.Up, var p, '-') => [new(p.Left(), Direction.Left), new(p.Right(), Direction.Right)],
    (Direction.Up, var p, '/') => [new(p.Right(), Direction.Right)],
    (Direction.Up, var p, '\\') => [new(p.Left(), Direction.Left)],
    (Direction.Up, var p, _) => [new(p.Up(), Direction.Up)],

    (Direction.Down, var p, '-') => [new(p.Left(), Direction.Left), new(p.Right(), Direction.Right)],
    (Direction.Down, var p, '/') => [new(p.Left(), Direction.Left)],
    (Direction.Down, var p, '\\') => [new(p.Right(), Direction.Right)],
    (Direction.Down, var p, _) => [new(p.Down(), Direction.Down)],

    (Direction.Left, var p, '|') => [new(p.Up(), Direction.Up), new(p.Down(), Direction.Down)],
    (Direction.Left, var p, '/') => [new(p.Down(), Direction.Down)],
    (Direction.Left, var p, '\\') => [new(p.Up(), Direction.Up)],
    (Direction.Left, var p, _) => [new(p.Left(), Direction.Left)],

    (Direction.Right, var p, '|') => [new(p.Up(), Direction.Up), new(p.Down(), Direction.Down)],
    (Direction.Right, var p, '/') => [new(p.Up(), Direction.Up)],
    (Direction.Right, var p, '\\') => [new(p.Down(), Direction.Down)],
    (Direction.Right, var p, _) => [new(p.Right(), Direction.Right)],
    
    _ => throw new Exception("unexpected combo")
};

In [6]:
var testInputString = """
.|...\....
|.-.\.....
.....|-...
........|.
..........
.........\
..../.\\..
.-.-/..|..
.|....-|.\
..//.|....
""";
var testInputLines = testInputString.Split('\n');
var testInputMap = new MirrorMap(testInputLines);
var testAnswer = ScanAll(testInputMap);

// Ultimately, in this example, 46 tiles become energized.
Console.WriteLine(testAnswer.Count);

46


In [7]:
var inputMap = new MirrorMap(inputLines);
var result = ScanAll(inputMap);

// The light isn't energizing enough tiles to produce lava; to debug the
// contraption, you need to start by analyzing the current situation. With the beam
// starting in the top-left heading right, how many tiles end up being energized?

var part1Answer = result.Count;
Console.WriteLine(part1Answer);

8146


In [8]:
// 8146 is correct!
Ensure(8146, part1Answer);

### --- Part Two ---

Puzzle description redacted as-per Advent of Code guidelines

You may find the puzzle description at: https://adventofcode.com/2023/day/16

In [10]:
IEnumerable<BeamPoint> GetAllStartPoints(MirrorMap map) {
    var topDown = Enumerable.Range(0, map.Cols).Select(col => new BeamPoint(new(0, col), Direction.Down));
    var bottomUp = Enumerable.Range(0, map.Cols).Select(col => new BeamPoint(new(map.Rows - 1, col), Direction.Up));
    var leftRight = Enumerable.Range(0, map.Rows).Select(row => new BeamPoint(new(row, 0), Direction.Right));
    var rightLeft = Enumerable.Range(0, map.Rows).Select(row => new BeamPoint(new(row, map.Cols - 1), Direction.Left));

    return topDown.Concat(bottomUp).Concat(leftRight).Concat(rightLeft);
}

In [11]:
// Using this configuration, 51 tiles are energized:

var part2TestAnswer = GetAllStartPoints(testInputMap).Select(sp => ScanAll(testInputMap, sp).Count()).Max();
Console.WriteLine(part2TestAnswer);

51


In [12]:
// Find the initial beam configuration that energizes the largest number of tiles; how many tiles are energized in that configuration?

var part2Answer = GetAllStartPoints(inputMap).Select(sp => ScanAll(inputMap, sp).Count()).Max();
Console.WriteLine(part2Answer);

8358


In [13]:
// 8358 is correct!
Ensure(8358, part2Answer);